In [7]:
from xml.etree import ElementTree
import re
import os
import pandas as pd


DIR= "/Users/hiteshochani/stat_nlp/project/blog/blogs"
error_count = 0
file_count = 0
parser1 = ElementTree.XMLParser(encoding="ascii")

header_x = "text\n"
header_y = "gender,age,industry,horroscope\n"
f1 = open('posts.csv','w')
f2 = open('labels.csv','w')

f1.write(header_x) 
f2.write(header_y) 


for file in os.listdir(DIR):
    metadata = file.split('.')[:-1]
    file_count +=1
    try:
        root = ElementTree.parse(DIR+'/'+file).getroot()
        text = ""
        for post in root.findall('post'):
            text+=post.text.strip()+' '
        f1.write(text.replace('\n', ' ').replace('\r', '')+'\n')
        f2.write(",".join(metadata) +'\n')
            
            
    except ElementTree.ParseError as p:
        error_count+=1
#         print(p, end="")
#         print(" " + file)
#         content = ""
#         with open(DIR+'/'+file, "rb") as fin:
#             content = fin.read()
#             content = re.sub(b'[^\x00-\x7F]+',b'', content)
#             content = re.sub(b'&[a-z]+;',b' ', content)

#         with open(DIR+'/'+file, "wb") as fout:
#                 foo = content.replace(b'&', b'and')
#                 fout.write(foo)
f1.close()
f2.close()
print(error_count)



6383


In [8]:

df_x = pd.read_csv("posts.csv", sep="wubbalubbadbdb")
df_y = pd.read_csv("labels.csv")
df_input_text = df_x[:25]
df_output = df_y[:25]

/Users/hiteshochani/anaconda3/envs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [9]:
#compute embeddings here
import torch
import torchtext.vocab as vocab
glove = vocab.GloVe(name='6B', dim=50)

In [10]:
#get_emb_func = lambda x: glove[x]
#frame.apply(f)
# # print(df_input_text.iloc[0][0])

# import string
# first_text = df_input_text.iloc[0][0]
# text = df_input_text.iloc[0][0].translate(str.maketrans('','',string.punctuation))



In [11]:
# len(first_text)

In [12]:
#just for one sentence, calculate embeddings and context vector.
import corenlp
import os
os.environ['CORENLP_HOME'] = "/Users/hiteshochani/stat_nlp/project/stanford-corenlp-full-2018-10-05"


In [57]:
#**********This is too slow**********
#**********only 100k characters are allowed, exception if more than that.******
#, its hitting a url in background,
#need to precompute and store

#get_tokens returns a list of list of tokens, i.e. all sentences in tokenized manner.
def get_tokens(input_row):
    all_tokens = []
    with corenlp.CoreNLPClient(annotators="tokenize ssplit".split()) as client:
        ann = client.annotate(input_row[0][:100000])

        for sentence in ann.sentence:
            tokens = [t.word for t in sentence.token]
            all_tokens.append(tokens)
    return(all_tokens)

get_tokens(df_input_text.iloc[0]) 

[['DESTINY',
  '...',
  'you',
  'might',
  'not',
  'say',
  'anything',
  'but',
  'i',
  'can',
  'hear',
  'you',
  'have',
  'chosen',
  'me',
  ',',
  'your',
  'life',
  'partner',
  'so',
  'have',
  'i',
  'dear',
  ',',
  'so',
  'have',
  'i',
  'dear',
  '...',
  '.'],
 ['my',
  'first',
  'dream',
  ',',
  'my',
  'first',
  'extreme',
  ',',
  'my',
  'first',
  'love',
  ',',
  'i',
  'was',
  'waiting',
  'for',
  'my',
  'DESTINY',
  '.'],
 ['what',
  'should',
  'i',
  'do',
  'with',
  'myself',
  ',',
  'tell',
  'me',
  'o',
  "'",
  'my',
  'heart',
  'what',
  'should',
  'i',
  'do',
  'with',
  'myself',
  ',',
  'tell',
  'me',
  '...',
  '.'],
 ['should', 'i', 'fly', ',', 'with', 'this', 'beautiful', 'nature', '.'],
 ['or', 'should', 'i', 'play', 'with', 'these', 'winds', '.'],
 ['should',
  'i',
  'try',
  'to',
  'reach',
  'the',
  'skies',
  ',',
  'or',
  'should',
  'i',
  'pray',
  'to',
  'the',
  'mother',
  'earth',
  '.'],
 ['what',
  'should',
  '

In [50]:
def get_gender_as_0_1(gender_str):
    if gender_str == "male":
        return 0
    else:
        return 1
#get_gender_as_0_1("female")

In [110]:
#***** PAY ATTENTION*****Removes unknown words****
def get_flat_list_of_word_emb(input_sentences):
    flat_list_of_words = []
    for sublist in input_sentences:
        for item in sublist:
            if(torch.all(torch.eq(glove[item], torch.zeros(50)))): #*****Removes unknown words****
                pass
            else:
                flat_list_of_words.append(glove[item])
    return flat_list_of_words


# flat_list = [glove[item].data for sublist in input_sentences for item in sublist]
flat_list = get_flat_list_of_word_emb(input_sentences)
flat_list

[tensor([ 0.0808, -0.4312,  0.1561, -0.5581,  0.4365, -0.3512, -1.1107,  0.1662,
         -0.4268,  0.1285, -0.3087,  0.5103, -1.1320, -0.1884,  1.2043,  0.1746,
          0.3722,  0.3214, -0.1399, -0.9825, -0.6658,  0.7520,  1.0343,  0.0061,
          0.9381, -1.7012, -1.0297,  0.5278,  1.0621, -0.9673,  3.3631,  0.7105,
          0.3020, -0.3352, -0.0163,  0.0915, -0.0084,  0.9456,  0.1803, -0.3841,
         -0.6449, -0.1697, -0.1266,  0.9894, -0.0839, -0.0883, -0.2243, -0.0282,
         -0.5100,  0.6169]),
 tensor([ 0.7689, -0.7430, -0.1578, -1.0023, -0.7347, -0.2382, -1.0935,  0.0854,
          0.0128,  0.0144, -0.1312, -0.1668, -0.3581, -0.0873,  0.3370,  0.1667,
         -0.3717, -0.2037,  0.0789, -0.2747,  1.3625, -0.0244,  0.1986, -0.3475,
          0.1777, -1.1291,  0.1086, -0.2126,  0.3317, -0.0062,  2.6137,  0.1479,
         -0.6207, -1.0393, -0.3569,  0.0328, -0.4797,  0.4316, -0.3326,  0.1399,
         -0.6605, -0.5965, -0.0988, -0.2597,  0.3245, -0.0581,  0.1732,  0.0928,

In [15]:
df_output.iloc[0]

gender               male
age                    16
industry          Student
horroscope    Sagittarius
Name: 4162441, dtype: object

In [56]:
df_input_text.iloc[0]

text    It's been a long time coming, but I have made ...
Name: 1, dtype: object

In [16]:
# # Parameters
# learning_rate = 0.01
# num_epochs = 10
# batch_size = 1
# display_step = 1

# # Network Parameters
# hidden_size = 100      # 1st layer and 2nd layer number of features
# input_size = 50 # assuming just word vocab as of now, later it should be 150
# num_classes = 2         # Categories: graphics, sci.space and baseball

In [52]:
# df_output.gender

In [ ]:
# def get_batch(df,i,batch_size):
#     batches = []
#     results = []
#     texts = df_input_text.text[i*batch_size:i*batch_size+batch_size]
#     categories = df_output.gender[i*batch_size:i*batch_size+batch_size]
#     for text in texts:
#         layer = np.zeros(total_words,dtype=float)
#         for word in text.split(' '):
#             layer[word2index[word.lower()]] += 1

#         batches.append(layer)

#     for category in categories:
#         index_y = -1
#         if category == 0:
#             index_y = 0
#         elif category == 1:
#             index_y = 1
#         else:
#             index_y = 2
#         results.append(index_y)


#     return np.array(batches),np.array(results)

In [25]:
import torch
from torch.autograd import Variable
import numpy as np
import pylab as pl
import torch.nn.init as init

In [26]:
dtype = torch.FloatTensor
# input_size, hidden_size, output_size = 50, 1, 1
epochs = 10
seq_length = 20
lr = 0.1

In [32]:
x = Variable(torch.Tensor([[1,2,3],[2,5,6]]), requires_grad=False)
# y = Variable(torch.Tensor(data[1:]).type(dtype), requires_grad=False)

In [67]:
#model
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class rcnn(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(rcnn, self).__init__()
        #****#,bias=True in below ??
        #??? should there be layer for wsl, wsr?
        self.layer_sl = nn.Linear(50,50) #semantic left
        self.layer_sr = nn.Linear(50,50) #semantic right
        self.layer_lc = nn.Linear(50,50) # left word and left context -> left context
        self.layer_rc = nn.Linear(50,50) # right word and right context -> right context
        self.layer_input = nn.Linear(150,100,bias=True) # lc, word, rc -> y hidden
        self.tanh = nn.tanh() 
        #******* what should be the maxPool size???
        self.layer_maxPool = nn.MaxPool1d(5)
        self.layer_linear = nn.Linear(20,2,bias=True)
        self.layer_softmax = nn.Softmax()
        

    def forward(left_word_emb,  current_word_emb , right_word_emb, left_context, right_context): #should the context be passed as well??
        #Assuming context is calculated inside the forward function.
        left_word_emb = self.self.layer_sl(left_word_emb)
        left_context = self.self.layer_lc(left_context)
        right_word_emb = self.layer_sr(right_word_emb)
        right_context = self.layer_rc(right_context)
        
        left_context = left_context + left_word_emb
        right_context = right_context + right_word_emb
        
        #concatenate here
        input = torch.cat((left_context,current_word_emb,right_context))
        input = self.tanh(self.layer_input(input))
        
        input = self.MaxPool1d(input)
        
        output = self.layer_softmax(self.layer_linear(input))
        return (output,left_context,right_context)


#      def forward(self, x):
#         out = self.layer_1(x)
#         out = self.relu(out)
#         out = self.layer_2(out)
#         out = self.relu(out)
#         out = self.output_layer(out)
#         return out

In [33]:
# def forward(left_word_emb,  current_word_emb , right_word_emb, left_context, right_context):
#     #Multiply by weights here
#     Wl * left_context + Wsl * prev_word
#     Wr * right_context + Wsr * next_word
#     #concatenate here
    
#     #Further layers
    
#   xh = torch.cat((input, right_context), 1)
#   context_state = torch.tanh(xh.mm(w1))
#   out = context_state.mm(w2)
#   return  (out, context_state)

In [61]:
for i in range(epochs):
  total_loss = 0
  context_state = None #Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=True)
  for j in range(0,df_input_text.text.size):
    #needs to be run for each word in a sentence
    #assume only first word for now
    input_sentences = get_tokens(df_input_text.iloc[j]) #all sentences, can do ravel instead 
    input = get_flat_list_of_word_emb(input_sentences)
    input = glove[ input_sentences[0][0].lower() ] # glove embedding of first word of first sentence
    print(input_sentences[0][0])
    print(input)
    target = get_gender_as_0_1( df_output.iloc[j].gender)
    print(target)
    
#     (pred, context_state) = forward(input, context_state, w1, w2)
#     loss = (pred - target).pow(2).sum()/2
#     total_loss += loss
#     loss.backward()
#     w1.data -= lr * w1.grad.data
#     w2.data -= lr * w2.grad.data
#     w1.grad.data.zero_()
#     w2.grad.data.zero_()
#     context_state = Variable(context_state.data)
  if i % 10 == 0:
     print("Epoch: {} loss {}".format(i, total_loss))

DESTINY
tensor([ 0.5473,  0.5169,  0.0295,  0.2992,  1.1377,  0.4791,  0.3222,  0.0666,
         0.6030,  0.2123,  0.6300,  0.6149, -0.7968,  0.7400,  0.0720,  0.3642,
         0.0197, -0.0513,  0.3029,  0.1033, -0.5545,  0.6291, -0.1391,  0.1858,
         0.6319, -0.7077, -1.5172, -0.4848,  0.4947, -0.4868,  1.1384,  0.4002,
        -0.7497, -0.1265, -0.1704, -0.3409, -0.0326, -0.4719, -0.2089, -0.8535,
        -0.1900, -1.1916,  0.0072, -0.1365, -0.5936,  0.6261,  0.3613, -0.5212,
        -0.3627, -0.1763])
0
It
tensor([ 0.6118, -0.2207, -0.1090, -0.0530,  0.5080,  0.3468, -0.3356, -0.1915,
        -0.0359,  0.1051,  0.0794,  0.2449, -0.4373, -0.3334,  0.5748,  0.6905,
         0.2971,  0.0907, -0.5499, -0.4618,  0.1011, -0.0202,  0.2848,  0.0435,
         0.4573, -2.0466, -0.5808,  0.6180,  0.6518, -0.5826,  4.0786, -0.2542,
        -0.1465, -0.3432, -0.2544, -0.4468,  0.1266,  0.2813,  0.1333, -0.3697,
         0.0501, -0.1006, -0.0179,  0.1114, -0.7180,  0.4910, -0.1000, -0.0437,


Age
tensor([-0.4385,  1.1905, -0.1163, -0.9594,  0.4589,  1.4150, -0.6760, -0.8434,
        -0.7357,  0.2872,  0.6475, -0.3622,  0.1149, -0.3402,  1.1601,  0.2720,
        -0.4529,  0.2052, -0.8096,  0.5534, -0.2132,  0.9178,  0.6683,  0.0559,
         0.3971, -0.9000, -1.1099, -0.9121,  0.3440,  0.4425,  2.9440,  0.2597,
         0.2412, -0.6596,  0.7578, -0.1379,  0.1987,  0.2505,  0.4246, -0.1881,
        -1.0105, -0.4946,  0.5689,  0.7545, -0.4724, -0.4388,  0.1331, -0.9140,
        -0.0862, -0.1799])
1
3
tensor([-0.3091,  0.8330,  0.8006,  0.5646,  0.4235,  0.6813, -0.1452, -0.3080,
        -0.7287, -0.4448, -0.0039, -0.3477, -0.2533, -0.4465,  0.5950, -0.6712,
        -0.4747, -0.1665, -1.7268,  0.4528,  0.2017, -0.7969,  1.2426,  0.3237,
        -0.7365, -0.4267,  0.9334, -0.5727,  0.3809, -0.4772,  3.5561,  0.3152,
        -0.3565,  0.6038,  0.4639, -0.2327,  0.5828,  0.2207,  0.7352, -0.5468,
         0.5000, -0.6742,  0.6379, -1.0965, -0.5135,  1.2085,  0.2982, -0.7595,
     

naked
tensor([-0.1580,  0.2656,  0.1606, -0.9311,  0.7477,  0.2287,  0.3393, -0.4760,
         0.5814, -0.0277, -0.1178, -0.0887, -0.3810,  0.8642,  0.7122,  0.1977,
         0.1785, -0.1221, -0.3938, -0.3840, -0.1105,  1.1061,  0.9808,  0.4778,
        -0.2546, -0.8355, -0.6992,  0.9598,  0.5519, -0.4876,  1.2361, -0.0867,
         0.2219, -0.2366, -0.8748,  0.7742,  0.3257, -0.5727, -0.4464, -0.9620,
         0.1870, -0.1359,  0.3137,  0.8085,  0.5537, -1.2751,  0.1431, -2.0834,
         0.2479, -0.7541])
0
Okay
tensor([ 0.1990, -0.7752, -0.1157, -0.3518,  0.4123, -0.6941,  0.1513,  0.3848,
        -0.2172,  0.0392,  0.3445,  0.6128, -0.5494,  0.0940,  0.8473,  0.9442,
         0.1213,  0.6523,  0.4049, -0.3232, -0.5773,  0.7318,  0.6052,  0.4832,
         1.1837, -0.5520, -0.5691,  0.8151,  0.5836, -0.9163,  0.8797,  0.9110,
        -0.1712,  1.0019, -0.1781, -0.7781,  0.8182,  0.1593,  0.3100, -0.1575,
        -0.2134, -0.4352,  0.1137,  0.3337, -0.3546, -0.0007, -0.0932,  0.4044,


KeyboardInterrupt: 

In [63]:
glove["mailto:abc@abc.com"] #unknowns come out as zeros

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [68]:
input_sentences = get_tokens(df_input_text.iloc[j]) 

In [89]:
input_sentences

[['I',
  "'ve",
  'begun',
  'a',
  'scary',
  'and',
  'daunting',
  'quest',
  ';',
  'becoming',
  'a',
  'Roman',
  'Catholic',
  '.'],
 ['Perhaps',
  'it',
  'does',
  "n't",
  'sound',
  'so',
  'scary',
  'to',
  'you',
  ',',
  'the',
  'reader',
  ',',
  'but',
  'to',
  'a',
  'simple',
  'Southern',
  'Baptist',
  'girl',
  'with',
  'an',
  'old',
  'and',
  'much',
  'loved',
  'aversion',
  'to',
  'all',
  'things',
  'Papal',
  ',',
  'this',
  'is',
  'little',
  'frightening',
  '.'],
 ['Let',
  "'s",
  'start',
  'at',
  'the',
  'beginning',
  '-',
  'the',
  'vital',
  'stats',
  'My',
  'name',
  'is',
  'Jaime',
  '.'],
 ['I',
  "'m",
  'a',
  '27',
  'year',
  'old',
  'woman',
  'who',
  'lives',
  'in',
  'Washington',
  'state',
  '.'],
 ['I', "'m", 'originally', 'from', 'Southern', 'California', '.'],
 ['My',
  'father',
  'is',
  'an',
  'excommunicated',
  'Catholic',
  'and',
  'my',
  'mother',
  'is',
  'Baptist',
  '.'],
 ['My',
  'father',
  'divorced

[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([ 0.0808, -0.4312,  0.1561, -0.5581,  0.4365, -0.3512, -1.1107,  0.1662,
         -0.4268,  0.1285, -0.3087,  0.5103, -1.1320, -0.1884,  1.2043,  0.1746,
          0.3722,  0.3214, -0.1399, -0.9825, -0.6658,  0.7520,  1.0343,  0.0061,
          0.9381, -1.7012, -1.0297,  0.5278,  1.0621, -0.9673,  3.3631,  0.7105,
          0.3020, -0.3352, -0.0163,  0.0915, -0.0084,  0.9456,  0.1803, -0.3841,
         -0.6449, -0.1697, -0.1266,  0.9894, -0.0839, -0.0883, -0.2243, -0.0282,
         -0.5100,  0.6169]),
 tensor([ 0.7689, -0.7430, -0.1578, -1.0023, -0.7347, -0.2382, -1.0935,  0.0854,
          0.0128,  0.0144, -0.1312, -0.1668, -0.3581, -0.0873,  0.3370,  0.1667,
         -0.3717, -0.2037,  0.0789, -0.2747,  1.3625, -0.0244,  0.1986, -0.3475,
          0.1

In [111]:
a = torch.randn(1,50) 

tensor([[-1.5664,  0.3804, -0.5083, -0.8332,  0.7734, -1.0524,  0.4457,  0.2317,
         -0.3636, -0.0728, -0.7751,  1.3358, -1.5732,  0.0252, -0.6261,  0.2340,
          0.0656, -0.0853,  0.4474,  1.0742, -0.8325, -1.5686, -0.3265, -0.3377,
          0.1114,  0.5564,  0.4771,  0.5472,  0.0387,  0.8708,  0.3609, -0.6512,
         -0.4466,  0.1583,  1.0877,  1.8305,  0.4919,  1.5223, -0.9237,  0.6287,
         -1.4961, -1.4586, -0.7093, -2.2030,  0.0562, -0.2424, -0.8021,  0.1808,
          0.3904, -0.4558]])